In [1]:
import SPOCK.long_term_scheduler as SPOCKLT
import SPOCK.short_term_scheduler as SPOCKST
import SPOCK.plots_scheduler as SPOCKplot
from astropy.time import Time
import pandas as pd
from astropy.utils import iers
import matplotlib.pyplot as plt
iers.IERS_A_URL  ='ftp://cddis.gsfc.nasa.gov/pub/products/iers/finals2000A.all'
import yaml
import sys
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import HTML, display, clear_output, Javascript
from plotly import graph_objs as go
from IPython.display import Image
import os
from astropy import units as u
import datetime
from astroplan import Observer
%matplotlib notebook
from astropy.coordinates import SkyCoord,EarthLocation,CartesianRepresentation
#from astroplan import download_IERS_A
#download_IERS_A()


# Tests App

In [2]:
observatory_widget = widgets.Combobox(
    # value='John',
    placeholder='Choose site',
    options=['SSO', 'SNO', 'Saint-Ex', 'TS','TN'],
    description='Site:',
    ensure_option=True,
    disabled=False
)

telescope_widget = widgets.Combobox(
    # value='John',
    placeholder='SSO telescope',
    options=['Io','Europa','Ganymede','Callisto'],
    description='Telescope:',
    ensure_option=True,
    disabled=False
)


start_date_widget =widgets.DatePicker(
    description='Start date',
    disabled=False
)
end_date_widget =widgets.DatePicker(
    description='End date',
    disabled=False
)

input_file_widget = widgets.Text(description='File',placeholder='input.csv')

run_widget = widgets.Button(
    value=False,
    description='Click me',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='check'
)

short_term_option_widget = widgets.Combobox(
    # value='John',
    placeholder='Choose site',
    options=['Follow-up','Special target with known start/end time',\
             'Special target (observe the most we can)','Monitoring'],
    description='What kind? :',
    ensure_option=True,
    disabled=False
)

start_time_widget =widgets.DatePicker(
    description='Start Time',
    disabled=False
)
end_time_widget =widgets.DatePicker(
    description='End Time',
    disabled=False
)

input_file_special_widget = widgets.Text(description='File',placeholder='input_short_term.csv')
 
target_widget = widgets.Combobox(
    # value='John',
    placeholder='Spxxx-xxxx',
    options=list(pd.read_csv('SPECULOOS_target_list_v2.txt',delimiter=' ')['Sp_ID']), 
    description='Target name:',
    ensure_option=True,
    disabled=False
)

plot_option_widget = widgets.Combobox(
    # value='John',
    placeholder='Choose a plot option',
    options=['Gant chart','Visibility  plot','Global gant chart'],
    description='What kind? :',
    ensure_option=True,
    disabled=False
)

plot_option_widget = widgets.Combobox(
    # value='John',
    placeholder='Choose a plot option',
    options=['Gant chart','Visibility  plot','Global gant chart'],
    description='What kind? :',
    ensure_option=True,
    disabled=False
)

chilean_ID_widget = widgets.Text(description='ID for ESO',placeholder='ID')
 
chilean_title_widget = widgets.Text(description='Title',placeholder='title')

chilean_PI_widget = widgets.Text(description='PI',placeholder='PI')

chilean_option_widgget = widgets.RadioButtons(
    options=['Make own plan', 'Use plan maker (fill fields below)'],
#    value='pineapple', # Defaults to 'pineapple'
#    layout={'width': 'max-content'}, # If the items' names are long
    description='Option:',
    disabled=False
)

chilean_nb_target_widget = widgets.BoundedIntText(
    value=1,
    min=1,
    max=3,
    step=1,
    description='# targets:',
    disabled=False
)



#g = interact(SPOCKplot.airmass_altitude_plot_given_target(name_observatory='Saint-Ex',day=days[0],target='Sp0804+6153',path_target_list=None))


def run_figure_widget():
    display(Javascript('Jupyter.notebook.execute_cells(3])'))
    interact(plt_target)

def run_all(b): 
    display(Javascript('Jupyter.notebook.execute_cells([3])'))

# ----------------------------------------------------------- #
    
long_term_tab = widgets.Accordion(children=[widgets.HBox([observatory_widget,telescope_widget]),\
                                        widgets.HBox([start_date_widget,end_date_widget]), \
                                        input_file_widget,\
                                       run_widget])
long_term_tab.set_title(0, 'Observatory')
long_term_tab.set_title(1, 'Date range')
long_term_tab.set_title(2, 'Input file')
long_term_tab.set_title(3, 'Run')

# ----------------------------------------------------------- #

short_term_tab = widgets.Accordion(children=[widgets.HBox([observatory_widget,telescope_widget]),\
                                        widgets.HBox([start_date_widget]), \
                                        input_file_special_widget,widgets.HBox([short_term_option_widget,start_time_widget,end_time_widget]),\
                                       run_widget])
short_term_tab.set_title(0, 'Observatory')
short_term_tab.set_title(1, 'Date range')
short_term_tab.set_title(2, 'Input file')
short_term_tab.set_title(3, 'Option')
short_term_tab.set_title(4, 'Run')

# ----------------------------------------------------------- #

plot_tab = widgets.Accordion(children=[input_file_widget,plot_option_widget,run_widget])
plot_tab.set_title(0, 'Input file')
plot_tab.set_title(1, 'Plot option')
plot_tab.set_title(2, 'Start/End date')
plot_tab.set_title(3, 'Run')

# ----------------------------------------------------------- #

chilean_tab = widgets.Accordion(children=[widgets.HBox([chilean_ID_widget,chilean_title_widget,chilean_PI_widget]),\
                                          widgets.VBox([chilean_option_widgget,widgets.HBox([start_time_widget,chilean_nb_target_widget,telescope_widget])]),\
                                          run_widget])
chilean_tab.set_title(0, 'Requiered')
chilean_tab.set_title(1, 'Plan maker options')
chilean_tab.set_title(2, 'Start/End date')
chilean_tab.set_title(3, 'Run')

# ----------------------------------------------------------- #



run_widget.on_click(run_all)


tab_nest = widgets.Tab()
tab_nest.children = [long_term_tab, short_term_tab,plot_tab,chilean_tab]
tab_nest.set_title(0, 'Long term scheduler ')
tab_nest.set_title(1, 'Short term scheduler')
tab_nest.set_title(2, 'Plots')
tab_nest.set_title(3, 'Chilean time')
tab_nest



In [3]:

if input_file_widget.value is '':
    input_file = 'input.csv'
else:
    input_file = input_file_widget.value

if tab_nest.selected_index == 0:
    if observatory_widget.value is '':
        sys.exit('ERROR: You must choose a site')
    else:
        if observatory_widget.value == 'SSO':
            obs = 1
        if observatory_widget.value == 'SNO':
            obs = 2
        if observatory_widget.value == 'Saint-Ex':
            obs = 3
        if observatory_widget.value == 'TS':
            obs = 4
        if observatory_widget.value == 'TN':
            obs = 5
    if start_date_widget.value is '' and end_date_widget.value is '':
        sys.exit('ERROR: The start and/or end date are not valid')
    else:
        start_date = Time(datetime.datetime.strptime(str(start_date_widget.value), '%Y-%m-%d')+datetime.timedelta(hours=15)).iso
        end_date = Time(datetime.datetime.strptime(str(end_date_widget.value), '%Y-%m-%d')+datetime.timedelta(hours=15)).iso
        schedule = SPOCKLT.Schedules()
        schedule.load_parameters(input_file,obs)
        if observatory_widget.value == 'SSO':
            schedule.telescope = str(telescope_widget.value)
        schedule.make_schedule(Altitude_constraint = 25, Moon_constraint = 30)
        display(schedule.target_table_spc[schedule.idx_first_target])
        display(schedule.target_table_spc[schedule.idx_second_target])

if tab_nest.selected_index == 1:
    if observatory_widget.value is '':
        sys.exit('ERROR: You must choose a site')
    else:
        if observatory_widget.value == 'SSO':
            obs = 1
        if observatory_widget.value == 'SNO':
            obs = 2
        if observatory_widget.value == 'Saint-Ex':
            obs = 3
        if observatory_widget.value == 'TS':
            obs = 4
        if observatory_widget.value == 'TN':
            obs = 5
    schedule = SPOCKST.Schedules()
    schedule.load_parameters(input_file,obs)
    
if tab_nest.selected_index == 2:   
    if plot_option_widget.value == 'Visiblity plot':
        day = Time(datetime.datetime.combine(start_date_widget.value,datetime.time(15,0,0)),format='datetime')
        @interact
        def plt_target(Site=['Saint-Ex','SSO','SNO','TS_La_Silla','TN_Oukaimeden'],Target=list(pd.read_csv('SPECULOOS_target_list_v2.txt',delimiter=' ')['Sp_ID']),day_widget = widgets.DatePicker(description='Pick a Date')):
            try:
                day = Time(datetime.datetime.combine(day_widget,datetime.time(15,0,0)),format='datetime')
                SPOCKplot.airmass_altitude_plot_given_target(name_observatory=Site,day=day,target=Target,path_target_list=None)

            except TypeError:
                print('ERROR: Choose a date ! ')
    if plot_option_widget.value == 'Global gant chart':
        SPOCKplot.gantt_chart_all(schedule.target_list)
        
    if plot_option_widget.value == 'Gant chart':
        @interact
        def plt_gant_chart(Telescope=['Europa','Io','Callisto','Ganymede','Artemis','Saint-Ex'],start_widget = widgets.DatePicker(description='Start Date'),end_widget = widgets.DatePicker(description='End Date')):
            try:
                day = Time(datetime.datetime.combine(day_widget,datetime.time(15,0,0)),format='datetime')
                SPOCKplot.gantt_chart(day,day+1,['Europa','Io','Callisto','Ganymede','Artemis','Saint-Ex'])
            except TypeError:
                print('ERROR: Choose a date ! ')

if tab_nest.selected_index == 3:
    chilean_target_name_widget = widgets.Text(description='Target name',placeholder='Chxxxx-xxxx')

    chilean_filter_widget = widgets.Combobox(
        # value='John',
        placeholder='Choose a filter',
        options=['I+z','r','z','i','Clear'],
        description='Filter :',
        ensure_option=True,
        disabled=False
    )


    chilean_texp_widget = widgets.BoundedIntText(
        value=20,
        min=10,
        max=2000,
        step=1,
        description='Exp time:',
        disabled=False
    )

    chilean_binnig_widget = widgets.BoundedIntText(
        value=1,
        min=1,
        max=10,
        step=1,
        description='binning:',
        disabled=False
    )

    chilean_ra_widgget = widgets.BoundedFloatText(
        value=0.0000,
        min=0.0,
        max=24,
        step=0.0001,
        description='RA (in °):',
        disabled=False
    )

    chilean_dec_widgget = widgets.BoundedFloatText(
        value=0.0000,
        min=-90.0,
        max=+90.0,
        step=0.0001,
        description='DEC (in °):',
        disabled=False
    )

    chilean_count_widget = widgets.BoundedIntText(
        value=5000,
        min=1,
        max=10000,
        step=1,
        description='Counts:',
        disabled=False
    )


    chilean_target_name2_widget = widgets.Text(description='Target name',placeholder='Chxxxx-xxxx')

    chilean_start_time_widget = widgets.Text(description='Start time',placeholder='2020-01-16 23:00:00')
    chilean_end_time_widget = widgets.Text(description='End time',placeholder='2020-01-17 03:00:00')

    chilean_filter2_widget = widgets.Combobox(
        # value='John',
        placeholder='Choose a filter',
        options=['I+z','r','z','i','Clear'],
        description='Filter :',
        ensure_option=True,
        disabled=False
    )


    chilean_texp2_widget = widgets.BoundedIntText(
        value=20,
        min=10,
        max=2000,
        step=1,
        description='Exp time:',
        disabled=False
    )

    chilean_binnig2_widget = widgets.BoundedIntText(
        value=1,
        min=1,
        max=10,
        step=1,
        description='binning:',
        disabled=False
    )

    chilean_ra2_widgget = widgets.BoundedFloatText(
        value=0.0000,
        min=0.0,
        max=24,
        step=0.0001,
        description='RA (in °):',
        disabled=False
    )

    chilean_dec2_widgget = widgets.BoundedFloatText(
        value=0.0000,
        min=-90.0,
        max=+90.0,
        step=0.0001,
        description='DEC (in °):',
        disabled=False
    )

    chilean_count2_widget = widgets.BoundedIntText(
        value=5000,
        min=1,
        max=10000,
        step=1,
        description='Counts:',
        disabled=False
    )


    run_chilean_widget = widgets.Button(
        value=False,
        description='Click me',
        disabled=False,
        button_style='', # 'success', 'info', 'warning', 'danger' or ''
        tooltip='Description',
        icon='check'
    )
    def run_chilean(b): 
        display(Javascript('Jupyter.notebook.execute_cells([4])'))



    if chilean_nb_target_widget.value == 1:
        chilean_tab = widgets.Accordion(children=[widgets.VBox([chilean_target_name_widget,widgets.HBox([chilean_ra_widgget,chilean_dec_widgget]),\
                                                  widgets.HBox([chilean_filter_widget,chilean_texp_widget,chilean_binnig_widget]),\
                                                  widgets.HBox([chilean_start_time_widget,chilean_end_time_widget]),\
                                                  chilean_count_widget]),\
                                                  run_chilean_widget])

        chilean_tab.set_title(0, 'First target')
        chilean_tab.set_title(1, 'Run')

    if chilean_nb_target_widget.value == 2:
        chilean_tab = widgets.Accordion(children=[widgets.VBox([chilean_target_name_widget,widgets.HBox([chilean_ra_widgget,chilean_dec_widgget]),
                                                  widgets.HBox([chilean_filter_widget,chilean_texp_widget,chilean_binnig_widget]),\
                                                  widgets.HBox([chilean_start_time_widget,chilean_end_time_widget]),\
                                                  chilean_count_widget ,telescope_widget]),\
                                                  widgets.VBox([chilean_target_name2_widget,widgets.HBox([chilean_ra2_widgget,chilean_dec2_widgget]),
                                                  widgets.HBox([chilean_filter2_widget,chilean_texp2_widget,chilean_binnig2_widget]),
                                                  widgets.HBox([chilean_start_time_widget,chilean_end_time_widget]),
                                                  chilean_count2_widget]),
                                                  run_chilean_widget])

        chilean_tab.set_title(0, 'First target')
        chilean_tab.set_title(1, 'Second target')
        chilean_tab.set_title(2, 'Run')

    run_chilean_widget.on_click(run_chilean)
    tab_nest = widgets.Tab()
    tab_nest.children = [chilean_tab]
    tab_nest.set_title(0, 'Plan maker ')
    display(tab_nest)


SystemExit: ERROR: You must choose a site

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3327: UserWarning:

To exit: use 'exit', 'quit', or Ctrl-D.



In [ ]:
from SPOCK.test_txtfiles import startup, startup_no_flats, Path_txt_files, flatexo_gany, flatexo_io,flatexo_euro
from SPOCK.test_txtfiles import first_target,target, flatdawn, biasdark, shutdown

date = Time(Time(chilean_start_time_widget.value),out_subfmt='date')
telescope = telescope_widget.value
counts = chilean_count_widget.value

target_chilean = pd.DataFrame({'Date':date,'Telescope':telescope,'Name':chilean_target_name_widget.value,'Start':chilean_start_time_widget.value,'End':chilean_end_time_widget.value,'RA':chilean_ra_widgget.value,\
                             'DEC':chilean_dec_widgget.value,'Filter':chilean_filter_widget.value,'texp':chilean_texp_widget.value,\
                             'Counts':chilean_count_widget.value,'#target':chilean_nb_target_widget.value},index=[0])
if chilean_nb_target_widget.value > 1:
    other_target = {'Date':date,'Telescope':telescope,'Name':chilean_target_name2_widget.value,'Start':chilean_start_time2_widget.value,'End':chilean_end_time2_widget.value,'RA':chilean_ra2_widgget.value,\
                                 'DEC':chilean_dec2_widgget.value,'Filter':chilean_filter2_widget.value,'texp':chilean_texp2_widget.value,\
                                 'Counts':chilean_count2_widget.value,'#target':4}
    target_chilean = target_chilean.append(other_target,ignore_index=True,sort=False)

if chilean_nb_target_widget.value > 2:
    other_target = {'Date':date,'Telescope':telescope,'Name':chilean_target_name3_widget.value,'Start':chilean_start_time3_widget.value,'End':chilean_end_time3_widget.value,'RA':chilean_ra3_widgget.value,\
                                 'DEC':chilean_dec3_widgget.value,'Filter':chilean_filter3_widget.value,'texp':chilean_texp3_widget.value,\
                                 'Counts':chilean_count3_widget.value,'#target':4}
    target_chilean = target_chilean.append(other_target,ignore_index=True,sort=False)
    

def chilean_time_plans(target_chilean):
    for i in range(0,len(target_chilean)):
        Path = './'
        coords = SkyCoord(frame='icrs', ra=target_chilean['RA'][i], dec=target_chilean['DEC'][i],unit=(u.deg,u.deg), obstime='2020-01-16T00:00:00')
        #t = Time(Time(date).iso, out_subfmt='date')
        location = EarthLocation.from_geodetic(-70.40300000000002*u.deg, -24.625199999999996*u.deg,2635.0000000009704*u.m)
        paranal = Observer(location=location, name="paranal", timezone="UTC")
        t_now = Time(Time(target_chilean['Date'][i]), out_subfmt='date').iso
        sun_set_jd = paranal.sun_set_time(Time(target_chilean['Date'][i]),which='next')
        sun_rise_jd = paranal.sun_rise_time(Time(target_chilean['Date'][i])+1,which='next')
        sun_set=paranal.sun_set_time(Time(target_chilean['Date'][i]),which='next')
        twilight_evening_nautic_jd = paranal.twilight_evening_nautical(Time(target_chilean['Date'][i]),which='nearest')
        twilight_evening_civil_jd = paranal.twilight_evening_civil(Time(target_chilean['Date'][i]),which='nearest')
        sun_rise=paranal.sun_rise_time(Time(target_chilean['Date'][i]),which='next')
        twilight_morning_nautic_jd=paranal.twilight_morning_nautical(Time(target_chilean['Date'][i]),which='nearest')
        twilight_morning_civil_jd=paranal.twilight_morning_civil(Time(target_chilean['Date'][i]),which='nearest')

        autofocus = None
        waitlimit = 600
        afinterval = 60
        count = counts

        date_start = Time(target_chilean['Start'][i]).iso
        date_end = Time(target_chilean['End'][i]).iso

        if 'Io' in target_chilean['Telescope'][i]:
            startup(t_now,target_chilean['Name'][i],sun_set.iso,date_start,Path)
        if 'Europa' in target_chilean['Telescope'][i]:
            startup(t_now,target_chilean['Name'][i],sun_set.iso,date_start,Path)
        if 'Ganymede' in target_chilean['Telescope'][i]:
            startup(t_now,target_chilean['Name'][i],sun_set.iso,date_start,Path)
        if 'Callisto' in target_chilean['Telescope'][i]:
            startup(t_now,target_chilean['Name'][i],sun_set.iso,date_start,Path)

        if i < len(target_chilean)-1:
            first_target(t_now,target_chilean['Name'][i],date_start,date_end,waitlimit,afinterval, autofocus,target_chilean['Counts'][i],target_chilean['Filter'][i],target_chilean['texp'][i],coords.ra.hms[0],coords.ra.hms[1],coords.ra.hms[2],coords.dec.dms[0],coords.dec.dms[1],coords.dec.dms[2],t_now,target_chilean['Name'][i+1],Path)

        if i == len(target_chilean)-1:
            target(t_now,target_chilean['Name'][i],date_start,date_end,waitlimit,afinterval, autofocus,target_chilean['Counts'][i],target_chilean['Filter'][i],target_chilean['texp'][i],coords.ra.hms[0],coords.ra.hms[1],coords.ra.hms[2],coords.dec.dms[0],coords.dec.dms[1],coords.dec.dms[2],None,Path)
            flatdawn(t_now,date_end,sun_rise.iso,Path)

    print('INFO: Plans are now in the ' + str(Path + t_now) + ' folder')
chilean_time_plans(target_chilean)
    

In [ ]:
make_your_own_widget = widgets.Textarea(
    value='; == Startup == \n\
    ;\n\
    ;\n\
    #waituntil 1, 2020/01/17 23:31:33\n\
    #domeopen\n\
    ;\n\
    #chill -60\n\
    ;\n\
    #waituntil 1, 2020/01/17 23:41:33\n\
    #duskflats Cal_flatexo.txt\n\
    ;\n\
    #quitat 2020/01/18 00:31:00\n\
    ;\n\
    #chain Obj_Sp0517-3349.txt\n\
    ;\n\
    ',
    placeholder='Type something',
    description='startup.txt:',
    disabled=False,
    layout={'height': '100%','width':'500px'}
)
box = widgets.VBox([make_your_own_widget], layout={'height': '500px','width':'600px'})
box


# LONG TERM SCHEDULER

In [5]:
# ---------------------- LONG TERM SCHEDULER ---------------------
obs = 2
schedule = SPOCKLT.Schedules()
schedule.load_parameters('./input.csv',obs)
#schedule.make_schedule(Altitude_constraint = 25, Moon_constraint = 30)


In [ ]:
display(schedule.target_table_spc[schedule.idx_first_target])
display(schedule.target_table_spc[schedule.idx_second_target])


In [ ]:
SPOCKLT.save_schedule('./input.csv',obs,save=True,over_write=True)

In [ ]:
schedule.telescope  = 'Artemis'
day = schedule.date_range[0]
SPOCKLT.make_np(day,4,schedule.telescope)
SPOCKLT.upload_plans(day, nb_days=4,telescope = schedule.telescope)

# SHORT TERM SCHEDULER

In [ ]:
with open('./input_short_term.csv', "r") as f:
    Inputs = yaml.load(f, Loader=yaml.FullLoader)
    df = pd.DataFrame.from_dict(Inputs['observatories'])
#::: observatory drowndown widget
observatory_w = widgets.Dropdown(
    options=df[1]['telescopes'],
    value='paranal_local',
    description='Observatory'
)
hbox1 = widgets.HBox([observatory_w])
display(hbox1)

In [ ]:
# ---------------------- SHORT TERM SCHEDULER ---------------------
obs = 1
schedule = SPOCKST.Schedules()
schedule.load_parameters('./input_short_term.csv',obs)

if schedule.use == 'follow_up':
    schedule.transit_follow_up('target_transit_follow_up.txt')
if schedule.use == 'special_start_end':
    input_name = 'Trappist-1b'
    schedule.special_target_with_start_end(input_name)
if schedule.use == 'special':
    input_name = 'Sp0938-2748'
    schedule.special_target(input_name)
if schedule.use == 'monitoring':
    input_name = 'Sp0755-2404'
    schedule.monitoring(input_name,5,61)

schedule.make_scheduled_table()
schedule.planification()
schedule.make_night_block()
SPOCKST.make_plans(day=schedule.day_of_night[0],nb_days=1,telescope=schedule.telescope)
schedule.scheduled_table

In [ ]:
SPOCKplot.airmass_plot_proposition(schedule.observatory.name,schedule.telescope,schedule.day_of_night[0])

In [ ]:
schedule.observatory.name = "Saint-Ex"

In [ ]:
schedule.scheduled_table_sorted


In [ ]:
SPOCKST.save_schedule('./input_short_term.csv',obs,save=True,over_write=True)

In [ ]:
day = schedule.day_of_night[0]
SPOCKST.make_np(day,1,schedule.telescope)
SPOCKST.upload_plans(day, nb_days=1,telescope = schedule.telescope)

# PLOT SCHEDULER

In [ ]:
# ---------------------- PLOT SCHEDULER ---------------------
day = Time('2020-01-09 15:00:00.000')


SPOCKplot.airmass_plot_saved(name_observatory='SSO',telescope='Io',day=day)
#SPOCKplot.airmass_altitude_plot_saved('SSO','Io',day)
#SPOCKplot.gantt_chart(day,day+1,['Artemis'])
#SPOCKplot.gantt_chart_all(schedule.target_list)
SPOCKplot.airmass_altitude_plot_given_target(name_observatory='SSO',day=day,target='Sp1048-1120',path_target_list=None)



In [6]:
SPOCKplot.gantt_chart_all(schedule.target_list)

In [ ]:
day = Time('2019-12-05 15:00:00.000')
SPOCKplot.gantt_chart(day,day+1,['Europa','Io','Callisto','Ganymede','Artemis','Saint-Ex'])

# Plots Meeting

In [ ]:
# ---------------------- LONG TERM SCHEDULER ---------------------
import matplotlib.pyplot as plt
from astropy.time import Time
from astroplan.plots import plot_sky
from astroplan import FixedTarget, Observer
import numpy as np
from astropy import units as u
import matplotlib.pyplot as plt
from astropy.coordinates import SkyCoord,EarthLocation,CartesianRepresentation
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
obs = 2
schedule = SPOCKLT.Schedules()
schedule.load_parameters('./input.csv',obs)
df = schedule.target_table_spc
df

In [ ]:
coords = SkyCoord(frame='icrs', ra=df['RA'], dec=df['DEC'],unit=(u.deg,u.deg), obstime='2020-01-16T00:00:00')
coord_cartesian = coords.represent_as(CartesianRepresentation) 
observed_on_going = np.where((df['nb_hours_surved'] != 0) & (df['nb_hours_surved'] <= 100))
observed_done = np.where((df['nb_hours_surved'] > 100))
to_be_done = np.where((df['nb_hours_surved'] ==0))
coords[0].ra.hms[0],coords[0].ra.hms[1],coords[0].ra.hms[2]

In [ ]:
halfpi, pi, twopi = [f*np.pi for f in [0.5, 1, 2]]

degs, rads = 180/pi, pi/180

# do radians first, then convert later

nstars = 2000

ran1, ran2 = np.random.random(2*nstars).reshape(2, -1)

RA  = twopi * (ran1 - 0.5)
dec = np.arcsin(2.*(ran2-0.5)) # Hey Barry Carter!

funcs = [np.cos, np.sin]

cosRA,  sinRA  = [f(RA)  for f in funcs]
cosdec, sindec = [f(dec) for f in funcs]

x = cosRA * cosdec
y = sinRA * cosdec
z = sindec

decs = (np.arange(11)-5) * halfpi / 6.
RAs  = (np.arange(12)-5) * halfpi / 6.

theta = np.linspace(0, twopi, 101)

costh, sinth = [f(theta) for f in funcs]
zerth = np.zeros_like(theta)

fig = plt.figure(figsize= [14, 14])

ax  = fig.add_subplot(1, 1, 1, projection='3d')

#ax.plot(x, y, z, '.k')

#for i in range(len(df)):
plt.plot(coord_cartesian.x, coord_cartesian.y, coord_cartesian.z,'.', color='gainsboro',alpha=0.9, markersize=6,zorder=2,label='To be done')
plt.plot(coord_cartesian[observed_on_going].x, coord_cartesian[observed_on_going].y, coord_cartesian[observed_on_going].z,'.', color='orange',alpha=0.9, markersize=6,zorder=3,label='On going')
plt.plot(coord_cartesian[observed_done].x, coord_cartesian[observed_done].y, coord_cartesian[observed_done].z,'.', color='red',alpha=0.9, markersize=6,zorder=4,label='Done')
    
    
ax.set_xlim(-1.1, 1.1)
ax.set_ylim(-1.1, 1.1)
ax.set_zlim(-1.1, 1.1)

ax.set_xlabel('x',fontsize=14)
ax.set_ylabel('y',fontsize=14)
ax.set_zlabel('z',fontsize=14)
ax.tick_params(axis="x", labelsize=12, labelcolor="k")
ax.tick_params(axis="y", labelsize=12, labelcolor="k")
ax.tick_params(axis="z", labelsize=12, labelcolor="k")
ax.view_init(elev=20, azim=47)

decs = (np.arange(11)-5) * halfpi / 6.
RAs  = (np.arange(12)-5) * halfpi / 6.
# lines of declination
xvals = costh * np.cos(decs)[:, None]
yvals = sinth * np.cos(decs)[:, None]
zvals = zerth + np.sin(decs)[:, None]
for x, y, z in zip(xvals, yvals, zvals):
    plt.plot(x, y, z, 'k',alpha=0.5, linewidth=0.8,zorder=1)

# lines of Right Ascention
xvals = costh * np.cos(RAs)[:, None]
yvals = costh * np.sin(RAs)[:, None]
zvals = sinth + np.zeros_like(RAs)[:, None]
for x, y, z in zip(xvals, yvals, zvals):
    plt.plot(x, y, z, 'k',alpha=0.5, linewidth=0.8,zorder=1)
plt.legend(fontsize='xx-large')

plt.savefig('./SPOCK_Figures/All_sky_cartesian_done_or not.pdf')

In [ ]:
halfpi, pi, twopi = [f*np.pi for f in [0.5, 1, 2]]

degs, rads = 180/pi, pi/180

# do radians first, then convert later

nstars = 2000

ran1, ran2 = np.random.random(2*nstars).reshape(2, -1)

RA  = twopi * (ran1 - 0.5)
dec = np.arcsin(2.*(ran2-0.5)) # Hey Barry Carter!

funcs = [np.cos, np.sin]

cosRA,  sinRA  = [f(RA)  for f in funcs]
cosdec, sindec = [f(dec) for f in funcs]

x = cosRA * cosdec
y = sinRA * cosdec
z = sindec

decs = (np.arange(11)-5) * halfpi / 6.
RAs  = (np.arange(12)-5) * halfpi / 6.

theta = np.linspace(0, twopi, 101)

costh, sinth = [f(theta) for f in funcs]
zerth = np.zeros_like(theta)


fig = plt.figure(figsize= [14, 14])

ax  = fig.add_subplot(1, 1, 1, projection='3d')

io = []
europa = []
ganymede = []
callisto = []
artemis = []
saint_ex = []
trappist = []
for i in range(len(df)):
    if 'Io' in df['telescope'][i]:
        io.append(i)
    if 'Europa' in df['telescope'][i]:
        europa.append(i)
    if 'Ganymede' in df['telescope'][i]:
        ganymede.append(i)
    if 'Callisto' in df['telescope'][i]:
        callisto.append(i)
    if 'Artemis' in df['telescope'][i]:
        artemis.append(i)
    if 'Saint-Ex' in df['telescope'][i]:
        saint_ex.append(i)
    if 'TRAPPIST' in df['telescope'][i]:
        trappist.append(i)

#for i in range(len(df)):
plt.plot(coord_cartesian.x, coord_cartesian.y, coord_cartesian.z,'.', color='gainsboro',alpha=0.5,markersize=8, linewidth=0.8,zorder=2,label='Remaining')
plt.plot(coord_cartesian[trappist].x, coord_cartesian[trappist].y, coord_cartesian[trappist].z,'.', color='purple',alpha=0.5, markersize=8,zorder=3,label='TRAPPIST')

plt.plot(coord_cartesian[io].x, coord_cartesian[io].y, coord_cartesian[io].z,'.', color='red',alpha=0.5, markersize=8,zorder=3,label='Io')
plt.plot(coord_cartesian[europa].x, coord_cartesian[europa].y, coord_cartesian[europa].z,'.', color='blue',alpha=0.5, markersize=8,zorder=4,label='Europa')
plt.plot(coord_cartesian[ganymede].x, coord_cartesian[ganymede].y, coord_cartesian[ganymede].z,'.', color='orange',alpha=0.5, markersize=8,zorder=4,label='Ganymede')
plt.plot(coord_cartesian[callisto].x, coord_cartesian[callisto].y, coord_cartesian[callisto].z,'.', color='turquoise',alpha=0.5, markersize=8,zorder=4,label='Callisto')

plt.plot(coord_cartesian[artemis].x, coord_cartesian[artemis].y, coord_cartesian[artemis].z,'.', color='green',alpha=0.5, markersize=8,zorder=4,label='Artemis')
plt.plot(coord_cartesian[saint_ex].x, coord_cartesian[saint_ex].y, coord_cartesian[saint_ex].z,'.', color='gold',alpha=0.5, markersize=8,zorder=4,label='Saint-Ex')


ax.set_xlim(-1.1,1.1)
ax.set_ylim(-1.1,1.1)
ax.set_zlim(-1.1,1.1)


decs = (np.arange(11)-5) * halfpi / 6.
RAs  = (np.arange(12)-5) * halfpi / 6.
# lines of declination
xvals = costh * np.cos(decs)[:, None]
yvals = sinth * np.cos(decs)[:, None]
zvals = zerth + np.sin(decs)[:, None]
for x, y, z in zip(xvals, yvals, zvals):
    plt.plot(x, y, z, 'k',alpha=0.5, linewidth=0.8,zorder=1)

# lines of Right Ascention
xvals = costh * np.cos(RAs)[:, None]
yvals = costh * np.sin(RAs)[:, None]
zvals = sinth + np.zeros_like(RAs)[:, None]
for x, y, z in zip(xvals, yvals, zvals):
    plt.plot(x, y, z, 'k',alpha=0.7, linewidth=0.8,zorder=1)

ax.set_xlabel('x',fontsize=14)
ax.set_ylabel('y',fontsize=14)
ax.set_zlabel('z',fontsize=14)
ax.tick_params(axis="x", labelsize=12, labelcolor="dimgrey")
ax.tick_params(axis="y", labelsize=12, labelcolor="dimgrey")
ax.tick_params(axis="z", labelsize=12, labelcolor="dimgrey")
ax.view_init(elev=1, azim=47)
plt.legend(fontsize='xx-large')
plt.savefig('./SPOCK_Figures/All_sky_cartesian_which_telescope.pdf')

import plotly.tools as tls

import matplotlib.animation as animation
from ipywidgets import *
import numpy as np
import matplotlib.pyplot as plt
#ani = animation.FuncAnimation(fig, fig_GIF,frames=2000, interval=20, blit=True)



In [ ]:
import matplotlib.pyplot as plt
from astropy.time import Time
from astroplan.plots import plot_sky
from astroplan import FixedTarget, Observer

location = EarthLocation.from_geodetic(-70.40300000000002*u.deg, -24.625199999999996*u.deg,2635.0000000009704*u.m)
observatory = Observer(location=location, name="SSO", timezone="UTC")
observed_on_going = np.where((df['nb_hours_surved'] != 0) & (df['nb_hours_surved'] <= 100))
observed_done = np.where((df['nb_hours_surved'] > 100))
#to_be_done = np.where((df['nb_hours_surved'] == 0))
targets_on_going = [FixedTarget(coord=coords[i],name=df['Sp_ID'][observed_on_going][i]) for i in range(len(df[observed_on_going]))]
targets_done = [FixedTarget(coord=coords[i],name=df['Sp_ID'][observed_done][i]) for i in range(len(df[observed_done]))]

observe_time = Time(['2020-01-16T00:00:00'])
not_done = {'color': 'gainsboro'}
done = {'color': 'red'}
on_going = {'color': 'orange'}

plt.figure()
plot_sky(targets, observatory, observe_time,style_kwargs=not_done)
plot_sky(targets_done, observatory, observe_time,style_kwargs=done)
#plot_sky(targets_on_going, observatory, observe_time,style_kwargs=on_going)
plt.legend(loc='center left', bbox_to_anchor=(1.25, 0.5))
plt.show()

In [ ]:
import numpy as np
from astropy import units as u
import matplotlib.pyplot as plt
from astropy.coordinates import SkyCoord,EarthLocation,CartesianRepresentation
plt.figure(figsize=(8,6))
plt.subplot(111, projection="aitoff")
ax = plt.gca()
#plt.title("Aitoff projection of our random data")
observed_on_going = np.where(((df['nb_hours_surved'] != 0) & (df['nb_hours_surved'] <= 100)))
observed_done = np.where((df['nb_hours_surved'] > 100))
to_be_done = np.where((df['nb_hours_surved'] == 0))

plt.plot(df['RA'][observed_on_going], df['DEC'][observed_on_going], 'o', markersize=5, alpha=0.8,color='orange',zorder=2,label='On going (#' + str(len(df['RA'][observed_on_going])) + ')' )
plt.plot(df['RA'][to_be_done], df['DEC'][to_be_done], 'o', markersize=5, alpha=0.3,color='gray',zorder=1,label='To be done (#' + str(len(df['RA'][to_be_done])) + ')')
plt.subplots_adjust(top=0.95,bottom=0.0)
ax.tick_params(axis="x", labelsize=12, labelcolor="k")
ax.tick_params(axis="y", labelsize=12, labelcolor="k")
#plt.legend(fontsize='xx-large',loc=4)
plt.grid(True)

plt.savefig('./SPOCK_Figures/All_sky_nb_hours_archived.pdf')
plt.show()

In [ ]:
import numpy as np
from astropy import units as u
import matplotlib.pyplot as plt
from astropy.coordinates import SkyCoord
plt.figure(figsize=(10,8))
plt.subplot(111, projection="aitoff")
ax = plt.gca()
#plt.title("Aitoff projection of our random data")
plt.grid(True)
io = []
europa = []
ganymede = []
callisto = []
artemis = []
saint_ex = []
trappist = []
for i in range(len(df)):
    if 'Io' in df['telescope'][i]:
        io.append(i)
    if 'Europa' in df['telescope'][i]:
        europa.append(i)
    if 'Ganymede' in df['telescope'][i]:
        ganymede.append(i)
    if 'Callisto' in df['telescope'][i]:
        callisto.append(i)
    if 'Artemis' in df['telescope'][i]:
        artemis.append(i)
    if 'Saint-Ex' in df['telescope'][i]:
        saint_ex.append(i)
    if 'TRAPPIST' in df['telescope'][i]:
        trappist.append(i)
#plt.plot(coords[io].ra, coords[io].dec, 'o', markersize=5, alpha=0.8,color='red',zorder=2,label='Io')
plt.plot(df['RA'][europa], df['DEC'][europa], 'o', markersize=5, alpha=0.8,color='blue',zorder=2,label='Europa' )
plt.plot(df['RA'][ganymede], df['DEC'][ganymede], 'o', markersize=5, alpha=0.8,color='orange',zorder=2,label='Ganymede')
plt.plot(df['RA'][callisto], df['DEC'][callisto], 'o', markersize=5, alpha=0.8,color='turquoise',zorder=2,label='Callisto')
plt.plot(df['RA'][artemis], df['DEC'][artemis], 'o', markersize=5, alpha=0.8,color='green',zorder=2,label='Artemis')
plt.plot(df['RA'][trappist], df['DEC'][trappist], 'o', markersize=5, alpha=0.8,color='purple',zorder=2,label='TRAPPIST')

plt.subplots_adjust(top=0.95,bottom=0.0)
ax.tick_params(axis="x", labelsize=14, labelcolor="k")
ax.tick_params(axis="y", labelsize=14, labelcolor="k")
plt.legend(fontsize='xx-large',loc=4)
plt.savefig('./SPOCK_Figures/All_sky_telescope.pdf')
plt.show()

In [ ]:
# ---------------------- LONG TERM SCHEDULER ---------------------
from astropy.table import Table
df_read = pd.read_csv('speculoos_target_list_webupdate.txt', delimiter=' ')
df = Table.from_pandas(df_read)

c = [SkyCoord(str(df['RA1'][i]) + ' ' +  str(df['RA2'][i]) + ' ' + str(round(df['RA3'][i])) + ' ' + str(df['DEC1'][i]) + ' ' + str(df['DEC2'][i]) + ' ' + str(round(df['DEC3'][i])), unit=(u.hourangle, u.deg)) for i in range(len(df))]

c

In [ ]:
c[list(observed_done[0])]

In [ ]:
import numpy as np
from astropy import units as u
import matplotlib.pyplot as plt
from astropy.coordinates import SkyCoord
plt.figure(figsize=(14,12))
plt.subplot(111, projection="aitoff")
ax = plt.gca()
#plt.title("Aitoff projection of our random data")
plt.grid(True)
observed_on_going = np.where(((df['nb_hours_surved'] != 0) & (df['nb_hours_surved'] <= 100)))
observed_done = np.where((df['nb_hours_surved'] > 100))
to_be_done = np.where((df['nb_hours_surved'] == 0))
plt.plot(c[observed_done].ra, c[observed_done].dec, 'o', markersize=5, alpha=0.8,color='red',zorder=2,label='Done (#' + str(len(df['RA'][observed_done])) + ')' )
plt.plot(c[observed_on_going].ra, c[observed_on_going].dec, 'o', markersize=5, alpha=0.8,color='orange',zorder=2,label='On going (#' + str(len(df['RA'][observed_on_going])) + ')' )
plt.plot(c[to_be_done].ra, c[to_be_done].dec, 'o', markersize=5, alpha=0.3,color='gray',zorder=1,label='To be done (#' + str(len(df['RA'][to_be_done])) + ')')
plt.subplots_adjust(top=0.95,bottom=0.0)
ax.tick_params(axis="x", labelsize=14, labelcolor="k")
ax.tick_params(axis="y", labelsize=14, labelcolor="k")
plt.legend(fontsize='xx-large',loc=4)
plt.show()

In [ ]:
program1 = np.where((df['prog']==1))
program2 = np.where((df['prog']==2))
program3 = np.where((df['prog']==3))
sso = []
artemis = []
saint_ex = []
trappist = []
for i in range(len(df)):
    if 'Io' in df['telescope'][i]:
        sso.append(i)
    if 'Europa' in df['telescope'][i]:
        sso.append(i)
    if 'Ganymede' in df['telescope'][i]:
        sso.append(i)
    if 'Callisto' in df['telescope'][i]:
        sso.append(i)
    if 'Artemis' in df['telescope'][i]:
        artemis.append(i)
    if 'Saint-Ex' in df['telescope'][i]:
        saint_ex.append(i)
    if 'TRAPPIST' in df['telescope'][i]:
        trappist.append(i)

plt.figure(figsize=(8,6))
ax = plt.gca()
#plt.plot(df['SpT'][observed_on_going],df['J'][observed_on_going],'.',markersize=8,color='orange',label='on going',zorder=2)
#plt.plot(df['SpT'][observed_done],df['J'][observed_done],'.',markersize=8,color='red',label='done',zorder=2)
plt.plot(df['SpT'],df['J'],'.',markersize=8,color='gainsboro',alpha=0.4,zorder=1,label='Target list stars')
#plt.plot(df['SpT'][to_be_done],df['J'][to_be_done],'.',markersize=8,color='brown',alpha=0.6,label='prog 1 (JWST)')
#plt.plot(df['SpT'][program1],df['J'][program1],'.',markersize=8,color='gold',alpha=0.9,label='prog 1 (JWST)')
#plt.plot(df['SpT'][program2],df['J'][program2],'.',markersize=8,color='blue',alpha=0.6,label='prog 2 (TESS)')
#plt.plot(df['SpT'][program3],df['J'][program3],'.',markersize=8,color='brown',alpha=0.6,label='prog 3 (UCD)')

plt.plot(df['SpT'][trappist],df['J'][trappist],'.',markersize=8,color='purple',alpha=0.6,label='TRAPPIST')
plt.plot(df['SpT'][sso],df['J'][sso],'.',markersize=8,color='skyblue',alpha=0.6,label='SSO')
plt.plot(df['SpT'][artemis],df['J'][artemis],'.',markersize=8,color='green',alpha=0.4,label='SNO')
plt.plot(df['SpT'][saint_ex],df['J'][saint_ex],'.',markersize=8,color='gold',alpha=0.4,label='Saint-Ex')


plt.legend(fontsize='xx-large',loc=4)
ax.set_xlabel('Spectral type',fontsize=14)
ax.set_ylabel('J magnitude',fontsize=14)
ax.tick_params(axis="x", labelsize=14, labelcolor="k")
ax.tick_params(axis="y", labelsize=14, labelcolor="k")
plt.savefig('Spt_vs_Jmag_telescope.pdf')
plt.show()

In [ ]:

fig, ax1 = plt.subplots()
ax1.plot(df['SNR_JWST_HZ_tr'][observed_on_going],df['J'][observed_on_going],'.',markersize=8,alpha=0.8,color='orange',label='on going',zorder=2)
ax1.plot(df['SNR_JWST_HZ_tr'][observed_done],df['J'][observed_done],'.',markersize=8,alpha=0.8,color='red',label='done',zorder=3)
ax1.plot(df['SNR_JWST_HZ_tr'],df['J'],'.',markersize=8,color='skyblue',alpha=0.4,zorder=1)
#ax1.plot(df['SNR_JWST_HZ_tr'][to_be_done],df['J'][to_be_done],'.',markersize=8,color='brown',alpha=0.6,label='prog 1 (JWST)')
#ax1.plot(df['SNR_JWST_HZ_tr'][program1],df['J'][program1],'.',markersize=8,color='gold',alpha=0.9,label='prog 1 (JWST)')
#ax1.plot(df['SNR_JWST_HZ_tr'][program2],df['J'][program2],'.',markersize=8,color='blue',alpha=0.6,label='prog 2 (TESS)')
#ax1.plot(df['SNR_JWST_HZ_tr'][program3],df['J'][program3],'.',markersize=8,color='brown',alpha=0.6,label='prog 3 (UCD)')

ax1.set_xlabel('SNR_JWST_HZ_tr',fontsize=14,color='skyblue')
ax1.set_ylabel('J magnitude',fontsize=14)
ax1.tick_params(axis="y", labelsize=14, labelcolor="k")
ax1.tick_params(axis='x', labelsize=14,labelcolor='skyblue')

ax2 = ax1.twiny()
ax2.plot(df['SNR_T1b'],df['J'],'.',markersize=8,color='seagreen',alpha=0.4,zorder=1)
ax2.plot(df['SNR_T1b'][observed_on_going],df['J'][observed_on_going],'.',markersize=8,color='orange',label='on going',zorder=3)
ax2.plot(df['SNR_T1b'][observed_done],df['J'][observed_done],'.',markersize=8,color='red',label='done',zorder=3)

ax2.set_xlabel('SNR_T1b ',fontsize=14,color="seagreen")
ax2.set_ylabel('J magnitude',fontsize=14)
ax2.tick_params(axis="x", labelsize=14, labelcolor="seagreen")



#plt.plot(df['SNR_JWST_HZ_tr'][trappist],df['J'][trappist],'.',markersize=8,color='purple',alpha=0.6,label='TRAPPIST')
#plt.plot(df['SNR_JWST_HZ_tr'][sso],df['J'][sso],'.',markersize=8,color='skyblue',alpha=0.6,label='SSO')
#plt.plot(df['SNR_JWST_HZ_tr'][artemis],df['J'][artemis],'.',markersize=8,color='green',alpha=0.4,label='SNO')
#plt.plot(df['SNR_JWST_HZ_tr'][saint_ex],df['J'][saint_ex],'.',markersize=8,color='gold',alpha=0.4,label='Saint-Ex')
ax2.tick_params(axis='x', labelcolor='green')
plt.legend(fontsize='x-large',loc=3)
fig.tight_layout()
plt.savefig('Spt_vs_SNRs_archived.pdf')
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Create some mock data
t = np.arange(0.01, 10.0, 0.01)
data1 = np.exp(t)
data2 = np.sin(2 * np.pi * t)

fig, ax1 = plt.subplots()

color = 'tab:red'
ax1.set_xlabel('time (s)')
ax1.set_ylabel('exp', color=color)
ax1.plot(t, data1, color=color)
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:blue'
ax2.set_ylabel('sin', color=color)  # we already handled the x-label with ax1
ax2.plot(t, data2, color=color)
ax2.tick_params(axis='y', labelcolor=color)

fig.tight_layout()  # otherwise the right y-label is slightly clipped
plt.show()